In [ ]:
!pip install transformers

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the FLAN-T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
context = """The Great Wall of China is one of the most famous landmarks in the world.
 It stretches over 13,000 miles and was originally built to protect against invasions from nomadic tribes.
  Construction of the wall began in the 7th century BC and continued until the 16th century.
   It was made primarily of earth, wood, and stone.
    The wall also served as a means of border control and trade route security.
     During the Ming dynasty, the wall was strengthened and expanded to its current form.
      Today, it is a UNESCO World Heritage Site and a symbol of China's historical strength and ingenuity.
       Visitors from around the globe come to see its impressive structure and learn about its history.
        The wall's construction involved millions of workers, including soldiers, peasants, and prisoners."""


In [ ]:
QUESTION_K = 40
QUESTION_P = 0.6
QUESTION_TEMP = 1.3

**Fact Based Questions:**

In [ ]:
def generate_unique_questions(context,
                              hyper_p=QUESTION_P,
                              hyper_k=QUESTION_K,
                              hyper_temp=QUESTION_TEMP ,
                              num_questions=2, maximum_rounds = 20):
    input_text = f"Generate diverse questions based on the following context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Use a set to store unique questions
    unique_questions = set()

    round = 0
    # Generate questions until we reach the desired number
    while len(unique_questions) < num_questions and round < maximum_rounds:
        round+=1
        outputs = model.generate(
            input_ids,
            max_new_tokens=50,
            temperature=hyper_temp,    # Higher temperature for increased diversity
            top_k=hyper_k,           # Limits sampling to the top-k most likely tokens
            top_p=hyper_p,          # Nucleus sampling, keeps tokens with cumulative probability ≥ p
            num_return_sequences=1,  # Return one sequence per generation
            do_sample=True,      # Ensure sampling is used
        )
        question = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # Add the question to the set if it's not a duplicate
        if question and question not in unique_questions:
            unique_questions.add(question)

    return list(unique_questions)


In [ ]:
import itertools

temp_arr_low = [0.5, 0.6, 0.7]
temp_arr_moderate = [0.8, 0.9, 1]
temp_arr_high = [1.1, 1.2, 1.3]

k_arr_low = [10, 25, 40]
k_arr_moderate = [50, 70, 100]
k_arr_high = [110, 150, 200]

p_arr_low = [0.5, 0.55, 0.6, 0.65]
p_arr_moderate = [0.7, 0.75, 0.8, 0.85]
p_arr_high = [0.9, 0.95, 1]


combinations = list(itertools.product(temp_arr_high, k_arr_low, p_arr_low))

questions=generate_unique_questions(context)
for question in questions:
  print("Question: " + question)


Question: How many years passed between the completion of the Great Wall of China and its construction?
Question: What are the main functions of the Great Wall of China?


best:hyper_p=0.6,hyper_k=40,hyper_temp=1.3

second best:hyper_p=0.7,hyper_k=30,hyper_temp=0.75

**Multiple Choices Questions**

In [ ]:
def generate_wrong_answer(context, question):
  input_text =f"Generate a wrong answer for question :{question} based on the following context:{context}"
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids
  outputs = model.generate(input_ids, max_new_tokens=50)
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return answer

**True or False Questions**